# Select top-N assets with good indicators

In [2]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore') #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; 
    else set it to False to turn off debugging mode '''
debug = True

In [72]:
import os
import sys
from datetime import datetime, date, timedelta
from pyspark.sql import functions as F

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from mining.modules.assets.etp import performIndex as indx
from utils.modules.etl.load import sparkDBwls as sdb
from utils.modules.ml.timeseries import rollingstats as stats
from utils.modules.etl.load import sparkNoSQLwls as nosql
from utils.modules.lib.spark import execSession as spark

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    indx = importlib.reload(indx)
    stats= importlib.reload(stats)
    nosql= importlib.reload(nosql)
    spark= importlib.reload(spark)
    sdb = importlib.reload(sdb)
    
__desc__ = "analyze crypto market capitalization time series data"
clsIndx =indx.Portfolio(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
clsSDB = sdb.SQLWorkLoads(desc=__desc__)
clsNoSQL=nosql.NoSQLWorkLoads(desc=__desc__)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional PERFORMINDEX-libraries in ETP-package of ASSETS-module imported successfully!
All packages in utils ml timeseries RollingStats imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional EXECSESSION-libraries in SPARK-package of LIB-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
sparkNoSQLwls Class initialization complete
dailyTopN Class initialization complete
performIndex Class initialization complete
sparkNoSQLwls Class initialization complete

Class initialization and load complete!


## Read top N mcap assets from tip sql db

In [54]:
_num_assets='3'
_date=datetime.strftime(date(2022,1,30),'%Y-%m-%dT00:00:00')
kwargs={}
_table='warehouse.mcap_past'
_query =f"select * from {_table} wmp where wmp.mcap_date = '{_date}' "+\
        f"and wmp.mcap_value > 10000 limit {_num_assets} "

_data = clsSDB.read_data_from_table(select=_query, **kwargs)
_data.show(n=1,vertical=True)

Wait a moment, retrieving data ...
-RECORD 0-----------------------------
 mcap_past_pk  | 75850                
 uuid          | 639753007cc473c58... 
 data_source   | coingecko            
 asset_name    | munt                 
 asset_symbol  | munt                 
 mcap_date     | 2022-01-30 00:00:00  
 mcap_value    | 4281638.502348270... 
 mcap_rank     | null                 
 created_dt    | 2023-02-14 12:22:... 
 created_by    | farmraider           
 created_proc  | wrangler_assets_e... 
 modified_dt   | 2023-02-21 14:45:... 
 modified_by   | farmraider           
 modified_proc | utils_etl_load_sp... 
 deactivate_dt | null                 
 log_ror       | 0.6602938723884690   
 simp_ror      | -0.0691060000000000  
only showing top 1 row



In [65]:
from pyspark.sql import functions as F
_assets=_data.select(F.col('mcap_date'),
                     F.col('asset_name'),
                     F.col('mcap_value'))\
                .distinct()
_portf=[]
for _asset in _assets.collect():
    _asset_dict={}
    _asset_dict={"date" : datetime.strftime(_asset[0],'%Y-%m-%dT00:00:00'),
                 "asset": _asset[1],
                 'mcap.weight': 1.0,
                 'mcap.value' : float(_asset[2]),
                }
    _portf.append(_asset_dict)
_portf

[{'date': '2022-01-30T00:00:00',
  'asset': 'munt',
  'mcap.weight': 1.0,
  'mcap.value': 4281638.50234827},
 {'date': '2022-01-30T00:00:00',
  'asset': 'aidoc',
  'mcap.weight': 1.0,
  'mcap.value': 50330.0135154841},
 {'date': '2022-01-30T00:00:00',
  'asset': 'mx',
  'mcap.weight': 1.0,
  'mcap.value': 178326244.574034}]

In [73]:
__idx_type__=['adx','sharp','rsi','mfi','beta']
_coll_dt=date(2022,1,30)
__val_col__="log_ror"
__name_col__='asset_name'
__date_col__='mcap_date'
__rf_assets__=['btc']
__rf_val_col__="simp_ror"
__rf_name_col__='asset_name'
__rf_date_col__='mcap_date'
_kwargs={
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_results=[]
for asset_portf in _portf:
    _idx_dict = clsIndx.get_index(
        portfolio=[asset_portf],
        asset_eval_date=_coll_dt,
        asset_name_col=__name_col__,
        asset_val_col =__val_col__,
        asset_date_col=__date_col__,
        index_type=__idx_type__,
        risk_free_assets=__rf_assets__,
        risk_free_name_col=__rf_name_col__,
        risk_free_val_col=__rf_val_col__,
        risk_free_date_col=__rf_date_col__,
        **_kwargs,
    )
    _results.append({asset_portf['asset']:_idx_dict})
# print("Completed index computation for %s"
#       % (str(_idx_dict)))
_results

Wait a moment, retrieving data ...
Wait a moment, retrieving data ...


Wait a moment, retrieving data ...
Wait a moment, retrieving data ...


Wait a moment, retrieving data ...
Wait a moment, retrieving data ...


[{'munt': {'sharp': -0.28348463831217335,
   'rsi': 1.0,
   'mfi': 1.0,
   'beta': -0.011394158192188886}},
 {'aidoc': {'sharp': 0.0113246408660233,
   'rsi': 1.0,
   'mfi': 1.0,
   'beta': 0.5857630438657931}},
 {'mx': {'sharp': 0.13875244455853572,
   'rsi': 1.0,
   'mfi': 1.0,
   'beta': 0.6915669539793301}}]